## data crawling

### primary link and ID

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

headers = {
    'Accept': 'text/html',
    'Accept-Language': 'en',
}

n = 2
path = 'data/steam_link.csv'

def getgamelist(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1, n):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            i = i.attrs
            i = i['href']
            link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
            ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
            linklist.append(link)
            IDlist.append(ID)
        print('completed '+str(pagenum)+' page, currently total '+str(len(linklist)))
    return linklist,IDlist

def getdf(n):
    linklist,IDlist = getgamelist(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)),
               columns =['Link', 'ID'])
    return df

if __name__ == "__main__":
    df = getdf(n)
    df.to_csv(path,index=False)

completed 1 page, currently total 25


### get the details in link

In [11]:
def gamename(soup): #name
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="apphub_AppName")
        k = str(a.text)
    return k

def gameprice(soup):#price
    try:
        a = soup.findAll(class_="discount_original_price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    except:
        a = soup.findAll(class_="game_purchase_price price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    return k

def taglist(soup):#tag list
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str(';'.join(list1))
    return list1

def getdate(soup):   #release date
    a = soup.find(class_="date")
    k = str(a.string)
    return k

def userreviewsrate(soup):  #rate
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return k

def developer(soup):   #developer
    a = soup.find(id="developers_list")
    k = str(a.a.string)
    return k

def findplatform(soup): #platform
    sysreq_tabs_div = soup.find('div', {'class': 'sysreq_tabs'})
    if sysreq_tabs_div:
        platforms_divs = sysreq_tabs_div.find_all('div', {'class': 'sysreq_tab'})
        return [div.get('data-os') for div in platforms_divs if div.get('data-os')]

    sysreq_contents_div = soup.find('div', {'class': 'sysreq contents'})
    if sysreq_contents_div:
        platforms_divs = sysreq_contents_div.find_all('div', {'class': 'game_area_sys_req sysreq_content active'})
        return [div.get('data-os') for div in platforms_divs if div.get('data-os')]

    return ['No platforms found']



def getgenre(soup): #genre
    genres_div = soup.find('div', {'id': 'genresAndManufacturer'})
    if genres_div:
        genre_b_tag = genres_div.find('b', text='Genre:')
        if genre_b_tag:
            genre_span = genre_b_tag.find_next_sibling('span')
            if genre_span:
                genres = genre_span.get_text(strip=True)
            else:
                genres = 'Genre not found'
        else:
            genres = 'Genre tag not found'
    else:
        genres = 'Genres and Manufacturer div not found'
    return genres


def getdetail(x):
    tag, date, rate, dev, name,price,genre,platform = ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '
    global count
    try:
        r = requests.get(x['Link'], headers=headers,timeout=10)
    except:
        print('server no reply 1')
        try:
            r = requests.get(x['Link'], headers=headers,timeout=10)
        except:
            print('server no reply 2')
            try:
                r = requests.get(x['Link'], headers=headers,timeout=10)
            except:
                print('server no reply 3')

    try:
        soup = BeautifulSoup(r.text, 'lxml')
        name = gamename(soup)
        tag = taglist(soup)
        date = getdate(soup)
        rate = userreviewsrate(soup)
        dev = developer(soup)
        price = gameprice(soup)
        genre = getgenre(soup)
        platform = findplatform(soup)
        print('complete: '+ name + str(x['ID']) +' num: %d'%count)
    except:
        print('not complete: '+ str(x['ID']) +' num: %d'%count)
        price = 'error'

    count += 1
    return name,price,tag,date,rate,dev,genre,platform

if __name__ == "__main__":
    df1 = pd.read_csv(path)
    count = 1
    df1['detail'] = df1.apply(getdetail, axis=1)
    df1['name'] = df1.apply(lambda x: x['detail'][0], axis=1)
    df1['price'] = df1.apply(lambda x: x['detail'][1], axis=1)
    df1['label'] = df1.apply(lambda x: x['detail'][2], axis=1)
    df1['release_date'] = df1.apply(lambda x: x['detail'][3], axis=1)
    df1['Recent_quantity_praise_rate'] = df1.apply(lambda x: x['detail'][4], axis=1)
    df1['developers'] = df1.apply(lambda x: x['detail'][5], axis=1)
    df1['genre'] = df1.apply(lambda x: x['detail'][6], axis=1)
    df1['platform'] = df1.apply(lambda x: x['detail'][7], axis=1)
    
    df1.to_csv('data/steam_detail.csv',index=False,encoding='utf-8-sig')
    print('Completed all')

C:\Users\12073\AppData\Local\Temp\ipykernel_46164\438019847.py:67: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  genre_b_tag = genres_div.find('b', text='Genre:')


complete: Lethal Company1966720 num: 1
complete: Counter-Strike 2730 num: 2
complete: Call of Duty®1938090 num: 3
complete: PUBG: BATTLEGROUNDS578080 num: 4
complete: Baldur's Gate 31086940 num: 5
complete: Remnant II1282100 num: 6
complete: Lost Ark1599340 num: 7
complete: EA SPORTS FC™ 242195250 num: 8
complete: Apex Legends™1172470 num: 9
complete: Lies of P1627720 num: 10
complete: F1® 232108330 num: 11
complete: Stellaris281990 num: 12
not complete: 271590 num: 13
complete: Victoria 3529340 num: 14
complete: MapleStory216150 num: 15
complete: War Thunder236390 num: 16
complete: Warframe230410 num: 17
complete: Tom Clancy's Rainbow Six® Siege359550 num: 18
complete: Cyberpunk 20771091500 num: 19
complete: ARK: Survival Ascended2399830 num: 20
complete: Sekiro™: Shadows Die Twice - GOTY Edition814380 num: 21
complete: NARAKA: BLADEPOINT1203220 num: 22
complete: Half-Life: Alyx546560 num: 23
complete: Football Manager 20242252570 num: 24
complete: Black Desert582660 num: 25
Completed

## data cleaning

In [1]:
import pandas as pd
import re

In [4]:
path = 'data/steam_original.csv'
df = pd.read_csv(path,index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35324 entries, 0 to 35323
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unnamed: 0                   35324 non-null  int64 
 1   Link                         35324 non-null  object
 2   ID                           35324 non-null  int64 
 3   detail                       35324 non-null  object
 4   name                         35324 non-null  object
 5   price                        35324 non-null  object
 6   label                        35324 non-null  object
 7   description                  35314 non-null  object
 8   recent_evalution             35324 non-null  object
 9   release_date                 35324 non-null  object
 10  Recent_quantity_praise_rate  35324 non-null  object
 11  developers                   35324 non-null  object
 12  comments                     34881 non-null  object
dtypes: int64(2), object(11)
memory 

In [22]:
df['price'].head(10)

0    $13.99
1    $13.99
2     $1.99
3     $0.99
4    $24.99
5     $4.99
6     $9.99
7     $2.97
8     $0.99
9     $4.99
Name: price, dtype: object

In [25]:
def price(x):
    try:
        pricenum = float(str(x['price']).replace('$',''))
    except:
        pricenum = 0
    return pricenum
df['price'] = df.apply(lambda x:price(x),axis=1)
df['price'] = pd.to_numeric(df['price'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36436 entries, 0 to 36435
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Link                         36436 non-null  object 
 1   ID                           36436 non-null  int64  
 2   detail                       36436 non-null  object 
 3   name                         36434 non-null  object 
 4   price                        36436 non-null  float64
 5   label                        36436 non-null  object 
 6   description                  36385 non-null  object 
 7   recent_evalution             36436 non-null  object 
 8   release_date                 36435 non-null  object 
 9   Recent_quantity_praise_rate  36436 non-null  object 
 10  developers                   36434 non-null  object 
 11  comments                     5797 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 3.6+ MB


In [30]:
def getreviewsnum(x):
    x1 = x['Recent_quantity_praise_rate']
    x2 = x['recent_evalution']
    if re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x1):
        num = re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x1).group(2)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x1):
        num = re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x1).group(2)
    elif re.search('(\d*) user reviews',x2):
        num = re.search('(\d*) user reviews',x2).group(1)
    else:
        num = '0'
    return num
def getreviewsrate(x):
    x = x['Recent_quantity_praise_rate']
    if re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x):
        rate = re.search('(\d*%) of the (\d*) user reviews for this game are positive in the past 30 days.',x).group(1)
    elif re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x):
        rate = re.search('(\d*%) of the (\d*) user reviews for this game are positive.',x).group(1)
    else :
        rate=''
    return rate
df['num_reviews']=df.apply(lambda x:getreviewsnum(x),axis=1)
df['positive_review_rate']=df.apply(lambda x:getreviewsrate(x),axis=1)
df

,Link,ID,detail,name,price,label,description,recent_evalution,release_date,Recent_quantity_praise_rate,developers,comments,评价数量,好评率,num_reviews,positive_review_rate
0,https://store.steampowered.com/app/1692000/,1692000,"('雨音スイッチ - Amane Switch -', '$13.99', 'Dating ...",雨音スイッチ - Amane Switch -,13.99,Dating Sim\nMultiple Endings\nStory Rich\nEmot...,Is there truly such a thing...as a happy endin...,Mostly Negative,"Aug 23, 2021",26% of the 15 user reviews for this game are p...,"CyberStep, Inc.",我每天都会来看你的，雨音，直到我死去\n看到这篇测评的兄弟姐妹们听我说：请不要购买此游戏，起...,15,26%,15,26%
1,https://store.steampowered.com/app/2001360/,2001360,"(""Mystical Riddles: Behind Doll’s Eyes Collect...",Mystical Riddles: Behind Doll’s Eyes Collector...,13.99,Hidden Object\nInteractive Fiction\nPuzzle\nVi...,What's hidden in an ordinary toy? Is it possib...,1 user reviews,"Sep 12, 2022",Need more user reviews to generate a score,Do Games Limited,NaN,1,,1,
2,https://store.steampowered.com/app/713590/,713590,"('Hidden Object - Food', '$1.99', 'Hidden Obje...",Hidden Object - Food,1.99,Hidden Object\nPuzzle\nClicker\nMinimalist\nRe...,This wonderful hidden object game gives you th...,Negative,"Sep 25, 2017",9% of the 11 user reviews for this game are po...,Boogygames Studios,NaN,11,9%,11,9%
3,https://store.steampowered.com/app/1759930/,1759930,"('Clarity of The Soul', '$0.99', 'Action\nActi...",Clarity of The Soul,0.99,Action\nAction-Adventure\nExploration\nFPS\n3D...,Clarity of the soul is a psychological horror ...,Negative,"Nov 12, 2021",18% of the 11 user reviews for this game are p...,VladEv1L,NaN,11,18%,11,18%
4,https://store.steampowered.com/app/1869870/,1869870,"('Cartomancy Anthology', '$24.99', 'Simulation...",Cartomancy Anthology,24.99,Simulation\nAction-Adventure\nArcade\nInteract...,Dive into the world of divination with the Car...,6 user reviews,"Aug 9, 2022",Need more user reviews to generate a score,Compact,"由22个不同游戏开发小团队制作的22个游戏,组合起来的独立游戏合集，每个游戏对应塔罗牌的一张...",6,,6,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36431,https://store.steampowered.com/app/1299170/,1299170,"('Ova Magica', 'error', 'Life Sim\nFarming Sim...",Ova Magica,0.00,Life Sim\nFarming Sim\nFishing\nCreature Colle...,Stardew Valley meets Pokémon! Experience a who...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,2023,No user reviews,ClaudiaTheDev,NaN,,,,
36432,https://store.steampowered.com/app/1301820/,1301820,"('Anomaly Hunter', 'PlayDemo', 'Point & Click\...",Anomaly Hunter,0.00,Point & Click\nHidden Object\nCasual\nExplorat...,Hidden object puzzler about time-traveling det...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,TBD,No user reviews,Hook Games,NaN,,,,
36433,https://store.steampowered.com/app/1302150/,1302150,"('Magicademia', 'error', 'Simulation\nMagic\nS...",Magicademia,0.00,Simulation\nMagic\nStrategy\nBuilding\nReal Ti...,Magicademia is a tycoon game in which you can ...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,TBA,No user reviews,STEREO GAMES S.A.,NaN,,,,
36434,https://store.steampowered.com/app/1302770/,1302770,"('Hero.EXE', 'Here.EXEPublicAlpha', 'Action\nA...",Hero.EXE,0.00,Action\nAdventure\nRPG\nIndie\nDeckbuilding\nC...,Hero.EXE is a card collecting action RPG. With...,\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t...,Coming Soon,No user reviews,Mystery Egg Games,NaN,,,,
